In [2]:
# Import required libraries
import pandas as pd
import numpy as np
from openpyxl import load_workbook

In [7]:
# Read the Excel file into a DataFrame, skipping the first row
df=pd.read_excel('./nsdp_delays_random.xlsx',skiprows=1)

In [8]:
# Print Data Frame
df

,Unnamed: 0,ctry,Yr,P/T,Cd,Dscrp,Yr.1,Jan,Feb,Mar,...,Sep,Q3,Oct,Nov,Dec,Q4,Unnamed: 23,Unnamed: 24,0,√ – SDDS requirement met;
0,1,AUS,2009,P,RLS,Real Sector ...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-9999.0,✕ – data were not disseminated;
1,2,AUS,2009,P,NAG00,National accounts ...,NaN,NaN,NaN,NaN,...,NaN,0,NaN,NaN,NaN,5,NaN,NaN,NaN,O – no ARC release dates provided;
2,3,AUS,2009,P,IND00,Production index ...,NaN,NaN,NaN,NaN,...,NaN,4,NaN,NaN,NaN,0,NaN,NaN,NaN,P – pending release; negative numeric entry – ...
3,4,AUS,2009,P,EMP00,Labor market: Employment ...,NaN,0,0,0,...,0,NaN,0,0,0,NaN,NaN,NaN,NaN,S – next to an observation indicates release r...
4,5,AUS,2009,P,UEM00,Labor market: Unemployment ...,NaN,0,0,0,...,0,NaN,0,0,7,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36140,42025,ZAF,2022,T,IIP00,International investment position ...,NaN,NaN,NaN,NaN,...,NaN,0,NaN,NaN,NaN,P,NaN,NaN,NaN,NaN
36141,42026,ZAF,2022,T,EXD00,External debt ...,NaN,NaN,NaN,NaN,...,NaN,5,NaN,NaN,NaN,P,NaN,NaN,NaN,NaN
36142,42027,ZAF,2022,T,EXR00,Exchange rates ...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
36143,42028,ZAF,2022,T,SOC,Socio-demographic Data ...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
# Select specific columns of interest
df = df[['ctry', 'Yr','P/T', 'Q1', 'Q2', 'Q3', 'Q4', 'Jan', 'Feb', 'Mar','Apr', 'May', 'Jun', 'Jul', 'Ago', 'Sep'
, 'Oct', 'Nov', 'Dec']]

In [10]:
# Convert numeric columns to numeric type
numeric_cols = ['Q1', 'Q2', 'Q3', 'Q4', 'Jan', 'Feb', 'Mar','Apr', 'May', 'Jun', 'Jul', 'Ago', 'Sep', 'Oct', 'Nov', 'Dec']

n= df[numeric_cols].apply(pd.to_numeric, errors='coerce')

In [11]:
# Filter rows where 'P/T' column is 'T' (timeliness data)
t = df['P/T'].astype(str)

t_data = df[df['P/T'].str.upper().eq('T')].copy()

In [14]:
#convert quarterly and monthly columns to numeric type
quarterly_cols = ['Q1', 'Q2', 'Q3', 'Q4']
t_data[quarterly_cols] = t_data[quarterly_cols].apply(pd.to_numeric, errors='coerce')
monthly_cols = ['Jan', 'Feb', 'Mar','Apr', 'May', 'Jun', 'Jul', 'Ago', 'Sep', 'Oct', 'Nov', 'Dec']
t_data[monthly_cols] = t_data[monthly_cols].apply(pd.to_numeric, errors='coerce')

In [16]:
# Replace invalid values 
t_data = t_data.replace(-9999,0)

In [17]:
# Clean Data (Fill NA with 0)
t_data.fillna(0, inplace=True)

In [18]:
# Calculate the sum of all relevant columns and create a new column 'final_col'
t_data['final_col'] = t_data['Q1'] + t_data['Q2'] + t_data['Q3'] + t_data['Q4'] + t_data['Jan'] + t_data['Feb'] + t_data['Mar'] + t_data['Apr'] +t_data['May'] + t_data['Jun'] + t_data['Jul'] + t_data['Ago'] + t_data['Sep'] + t_data['Oct'] + t_data['Nov'] + t_data['Dec']

In [19]:
#Now keeping only the relevant columns
final_df_cols = ['ctry','Yr','final_col']

In [20]:
# Retrieve final columns
final_df = t_data[final_df_cols]

In [21]:
# Aggregating the final dataframe
Grouped_data = final_df.groupby(['ctry','Yr']).mean()

In [22]:
Grouped_data

final_col
ctry Yr             
ARG  2013   5.185185
     2015   7.666667
     2016   5.629630
     2017   4.333333
     2018   5.370370
...              ...
ZAF  2018   6.888889
     2019   5.814815
     2020   7.814815
     2021   3.444444
     2022   5.851852

[640 rows x 1 columns]

In [23]:
# Load the Excel file
# Give full path of xlsx file and give full permissions to file
excel_file = './nsdp_delays_random.xlsx'
book = load_workbook(excel_file)
writer = pd.ExcelWriter(excel_file, engine='openpyxl') 
writer.book = book
Grouped_data.to_excel(writer, sheet_name='Aggregated Data', index=True)

# Save the changes
writer.save()
